In [1]:
# import libraries
from scipy import ndimage
from scipy import misc
import matplotlib.pyplot as plt
import glob
import numpy as np
from keras.utils import to_categorical
# import necessary building blocks for CNN
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout
from keras.layers.advanced_activations import LeakyReLU

Using TensorFlow backend.


In [2]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler
import re
from sklearn.model_selection import train_test_split

In [3]:
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping

## Read images, convert to Numpy

In [4]:
i = 0
images = []
for image_path in glob.glob("C:/Users/mvideo/Documents/Fax/data_2images_v2_27im/*.jpg"):
    #print(image.shape)
    #print(image.dtype)
    # put into np.array
    image = misc.imread(image_path)
    images.append(np.asarray( image, dtype="int32" ))
    # visualize images
    #plt.imshow(image)
    #plt.show()
X = np.concatenate([image[np.newaxis] for image in images])
print(type(X))
print(X.shape)
# (960, 1280, 3) 960, 1280 - количество пикселей. 3- rgb

<class 'numpy.ndarray'>
(27, 960, 1280, 3)


In [5]:
# Read the names of the files into y y.shape = (71,1).
labels = []
for image_path in glob.glob("C:/Users/mvideo/Documents/Fax/data_2images_v2_27im/*.jpg"):
    temp = str(image_path)[52:]
    temp = temp[:-6]
    temp = re.sub('_', '', temp)
    labels.append(int(temp))
y = np.asarray([x-1 for x in labels])
#y = [x-1 for x in y]
print(type(y))
#print(y.shape)

<class 'numpy.ndarray'>


In [6]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 2])

In [7]:
# Crop images from 960*1280 to 400*400
IMG_SIZE = 400
crop_coord1 = int(0.5*(X.shape[1] - IMG_SIZE))
crop_coord2 = int(0.5*(X.shape[2] - IMG_SIZE))
X_crop = X[:,crop_coord1:(crop_coord1+IMG_SIZE),crop_coord2:(crop_coord2+IMG_SIZE),:]
#cropped_img = slice1[:,crop_coord2:,:]

Normaly to train model we need dozens of thousands of images
Train samples: (50000, 32, 32, 3) (50000, 1)
Test samples: (10000, 32, 32, 3) (10000, 1)

## Prepare data
Normalize inputs. 
COnvert class labels to one-hot encoded vectors. Use keras.utils.to_categorical

In [50]:
# reshape image to normalize it in Standard Scaler (2d only authorized)
#X_resh = X_crop.reshape((2,IMG_SIZE*IMG_SIZE,3))

In [8]:
# normalize inputs
X_red = X_crop/255
y_cat = to_categorical(y, num_classes=len(np.unique(y)))

In [9]:
# SPlit the data
X_train, X_val, y_train, y_val = train_test_split(X_red, y_cat, test_size=0.2, random_state=42)
print(X_train.shape,' ',y_train.shape)
print(X_val.shape,' ',y_val.shape)

(21, 400, 400, 3)   (21, 3)
(6, 400, 400, 3)   (6, 3)


## Define CNN architecture

In [10]:
def make_model():
    model = Sequential()
    # 1st conv 
    model.add(Conv2D(16,(3,3),padding="same",activation='relu', input_shape=(400, 400, 3)))  # first layer needs to define "input_shape"
    model.add(BatchNormalization(axis=1)) # accelerate training
    model.add(LeakyReLU(0.1))

    # 2nd conv
    model.add(Conv2D(32,(3,3),padding="same",activation='relu')) 
    model.add(BatchNormalization(axis=1)) # accelerate training
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 3rd conv
    model.add(Conv2D(32,(3,3),padding="same",activation='relu'))  
    model.add(BatchNormalization(axis=1)) # accelerate training
    model.add(LeakyReLU(0.1))

    # 4rd conv
    model.add(Conv2D(64,(3,3),padding="same",activation='relu'))  
    model.add(BatchNormalization(axis=1)) # accelerate training
    model.add(LeakyReLU(0.1))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    # 1st dense layer
    model.add(Flatten())
    model.add(Dense(256))
    model.add(LeakyReLU(0.1))
    model.add(Dropout(0.5))

    # 2nd dense layer
    model.add(Dense(len(np.unique(y))))  # the last layer with neuron for each class
    model.add(Activation("softmax"))  # output probabilities

    return model

In [11]:
# describe model
#clear_session()  # clear default graph
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 400, 400, 16)      448       
_________________________________________________________________
batch_normalization_1 (Batch (None, 400, 400, 16)      1600      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 400, 400, 16)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 400, 400, 32)      4640      
_________________________________________________________________
batch_normalization_2 (Batch (None, 400, 400, 32)      1600      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 400, 400, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 200, 200, 32)      0         
__________

## Train model and Evaluate model

In [12]:
# continue to train model from checkpoint
from keras.models import load_model
#from keras.utils import ModelSaveCallback
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping


In [13]:
#INIT_LR = 5e-3  # initial learning rate
BATCH_SIZE = 10
EPOCHS = 5
model = make_model()  # define our model
last_finished_epoch = None
num_epochs = 25

# prepare model for fitting (loss, optimizer, etc)
model.compile(
    loss='categorical_crossentropy',  # we train 10-way classification
    optimizer='sgd',  # for SGD
    metrics=['mae', 'acc']  # report accuracy during training
)


In [15]:
from keras import utils as np_utils
# fit model
model.fit(
    X_train, y_train,  # prepared data
    batch_size=BATCH_SIZE,
    epochs=num_epochs,
    callbacks = [ModelCheckpoint('weights.{epoch:02d}-{val_acc:.2f}.hdf5',
                                monitor='val_loss', verbose=1, 
                                save_best_only=False, 
                                save_weights_only=False, 
                                mode='auto', 
                                period=1),
                EarlyStopping(monitor='val_loss', patience=8)],
    validation_data=(X_val, y_val),
    shuffle=True,
    verbose=1,
    initial_epoch=last_finished_epoch or 0
)

Train on 21 samples, validate on 6 samples
Epoch 1/25
21/21 [==============================] - 97s 5s/step - loss: 4.3793 - mean_absolute_error: 0.2222 - acc: 0.6667 - val_loss: 2.6863 - val_mean_absolute_error: 0.1111 - val_acc: 0.8333

Epoch 00001: saving model to weights.01-0.83.hdf5
Epoch 2/25
21/21 [==============================] - 85s 4s/step - loss: 6.1402 - mean_absolute_error: 0.2540 - acc: 0.6190 - val_loss: 2.6863 - val_mean_absolute_error: 0.1111 - val_acc: 0.8333

Epoch 00002: saving model to weights.02-0.83.hdf5
Epoch 3/25
21/21 [==============================] - 87s 4s/step - loss: 6.1402 - mean_absolute_error: 0.2540 - acc: 0.6190 - val_loss: 2.6863 - val_mean_absolute_error: 0.1111 - val_acc: 0.8333

Epoch 00003: saving model to weights.03-0.83.hdf5
Epoch 4/25
21/21 [==============================] - 82s 4s/step - loss: 5.6638 - mean_absolute_error: 0.2540 - acc: 0.6190 - val_loss: 1.1921e-07 - val_mean_absolute_error: 2.3212e-09 - val_acc: 1.0000

Epoch 00004: saving

In [18]:
# save weights to file
model.save_weights('weights_27images.h5')

In [ ]:
# load weights from file (can be called without model.fit)
model.load_weights('weights_27images.h5')